In [1]:
from agents.agent import Agent
from load_models import ModelLoader
from prompt_processing import PromptProcessing
from tools.tools import ToolExecutor, ToolRegistry
from tools.tools import EquationSolverTool, SymbolicMathTool, MatrixToolAgent, StatisticsToolAgent

registry = ToolRegistry()
registry.register(EquationSolverTool())
registry.register(SymbolicMathTool())
registry.register(MatrixToolAgent())
registry.register(StatisticsToolAgent())

executor = ToolExecutor(registry=registry)

loader = ModelLoader()

model, tokenizer = loader.load()

llm = loader.create_llm()


d:\Forum\Forum\AI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 290/290 [00:00<00:00, 291.05it/s, Materializing param=model.norm.weight]                              
d:\Forum\Forum\AI\venv\Lib\site-packages\peft\peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self

In [8]:
question = "Find derivative of x ** 4"

p_proc = PromptProcessing()

p_proc.load(filename="system_prompts.json")

SYSTEM_PROMPT = p_proc.get_system_prompt()

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": question}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

outputs = model.generate(
    **tokenizer(prompt, return_tensors="pt").to(model.device),
    max_new_tokens=200,
    do_sample=False,   
    temperature=0.0   
)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
tool_call = p_proc.extract_assistant_json(text)




In [10]:
result = executor.execute(tool_call)
result

4*x**3